## [E14]_Recommendation

## 목차
[0. data load and preprocessing](#-0.-data-load-and-preprocessing)

# 0. data load and preprocessing

In [1]:
import pandas as pd
import numpy as np
import os

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목 확인 
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
data = pd.merge(ratings,movies,how = 'left',on='movie_id')

In [8]:
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
5,1,1197,3,978302268,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
6,1,1287,5,978302039,Ben-Hur (1959),Action|Adventure|Drama
7,1,2804,5,978300719,"Christmas Story, A (1983)",Comedy|Drama
8,1,594,4,978302268,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
9,1,919,4,978301368,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


In [9]:
data['user_id'].nunique()

6039

In [10]:
data['title'].nunique()

3628

In [11]:
data['movie_id'].nunique()

3628

In [12]:
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(50)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [13]:
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [14]:
user_median = data.groupby('user_id')['counts'].mean()
user_median.describe()

count    6039.000000
mean        4.025078
std         0.275632
min         3.000000
25%         3.833333
50%         4.023622
75%         4.212225
max         5.000000
Name: counts, dtype: float64

In [15]:
data = data[['user_id','title','counts']]

In [16]:
len(data['user_id'].unique())

6039

In [17]:
max(data['user_id'])

6040

In [18]:
my_favorite = ['Terminator 2: Judgment Day (1991)' , 'Silence of the Lambs, The (1991)' ,'Matrix, The (1999)' ,'Jurassic Park (1993)' ,'Sixth Sense, The (1999)']

my_playlist = pd.DataFrame({'user_id': [max(data['user_id']+1)]*5, 'title': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

     # 잘 추가되었는지 확인해 봅시다.

data.reset_index(drop=True,inplace=True)

In [19]:
data

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,1,James and the Giant Peach (1996),3
2,1,My Fair Lady (1964),3
3,1,Erin Brockovich (2000),4
4,1,"Bug's Life, A (1998)",5
...,...,...,...
836478,6041,Terminator 2: Judgment Day (1991),5
836479,6041,"Silence of the Lambs, The (1991)",5
836480,6041,"Matrix, The (1999)",5
836481,6041,Jurassic Park (1993),5


In [20]:
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}


In [21]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')


temp_title_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
836478,6039,92,5
836479,6039,121,5
836480,6039,124,5
836481,6039,107,5


In [22]:
movie_count[movie_count<2]

title
1-900 (1994)                                   1
24-hour Woman (1998)                           1
Aiqing wansui (1994)                           1
Alley Cats, The (1968)                         1
American Strays (1996)                         1
                                              ..
Voyage to the Beginning of the World (1997)    1
War at Home, The (1996)                        1
Wend Kuuni (God's Gift) (1982)                 1
Wooden Man's Bride, The (Wu Kui) (1994)        1
Zachariah (1971)                               1
Name: user_id, Length: 119, dtype: int64

In [23]:
only_one = movie_count[movie_count<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

119,836483
Ratio of only_one over all data is 0.01%


In [24]:
num_user = data['user_id'].nunique()
num_moive = data['title'].nunique()

In [25]:
num_user

6040

In [26]:
num_moive

3628

In [27]:
max(data['title'])

3627

In [28]:
max(data['user_id'])

6039

In [29]:
num_user = data['user_id'].nunique()
num_moive = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_moive))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [30]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [31]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [32]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [33]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [34]:
alien = movie_to_idx['Alien (1979)']
my_vector, alien_vector = als_model.user_factors[6039], als_model.item_factors[alien]


In [35]:
my_vector

array([ 1.13647306e+00,  4.29189980e-01,  9.94689345e-01,  4.62800711e-01,
       -4.15345579e-01,  7.95868993e-01,  8.03689063e-01,  3.33841890e-02,
       -5.11322916e-01,  2.04940140e-01,  4.28391159e-01,  4.70457762e-01,
        6.76926851e-01, -2.76667804e-01,  9.62788414e-04,  1.36211365e-01,
       -1.28576949e-01,  1.02065706e+00,  1.52856052e-01,  2.69746315e-03,
       -5.24753630e-01, -3.37118357e-01, -4.84834939e-01, -1.14072151e-01,
       -4.08917189e-01, -6.17614508e-01, -3.88140172e-01,  6.71771407e-01,
       -6.76583648e-01,  7.18323469e-01,  9.49812472e-01, -2.71261722e-01,
        3.01689506e-01, -7.61504650e-01, -2.36649722e-01,  9.83297750e-02,
        3.47282052e-01,  1.95288017e-01, -5.20102382e-01,  2.74814785e-01,
        4.13280845e-01, -4.46002930e-01,  4.78833228e-01,  2.46345878e-01,
        4.31464136e-01,  7.54199624e-01,  1.16903834e-01,  3.64145964e-01,
        8.29952836e-01,  8.09801698e-01,  4.80681330e-01, -9.26585376e-01,
       -8.07979763e-01,  

In [36]:
alien_vector

array([-0.00581362,  0.00954592,  0.03371157,  0.01611475,  0.00054552,
        0.00450949,  0.00028307,  0.02782761, -0.0129029 ,  0.00827279,
        0.03546787,  0.01186229,  0.0042665 ,  0.00931898,  0.01113564,
        0.03278245,  0.00372096, -0.00476462, -0.00269237, -0.02454954,
        0.01125039,  0.02161596, -0.02492649,  0.02182725,  0.00330224,
       -0.01374836,  0.00991175,  0.01160949,  0.0067239 , -0.00067749,
        0.01737385, -0.01874331,  0.02243117,  0.00216441,  0.01863201,
        0.00675665, -0.00304848,  0.01447142,  0.03115856,  0.03127692,
        0.03531517, -0.00138347,  0.00522153, -0.00659285, -0.01280028,
        0.02584352,  0.03934997,  0.01642822,  0.00748258, -0.00831047,
       -0.04364627,  0.00849015, -0.01409939, -0.04243765,  0.01640431,
        0.01957951, -0.00438314,  0.01147793,  0.02180956,  0.01360709,
        0.01576558,  0.0202051 , -0.00074342,  0.0218972 , -0.00745372,
        0.02980737,  0.00491402,  0.010572  ,  0.00470102, -0.00

In [37]:
np.dot(my_vector, alien_vector)

0.11265808

In [38]:
rec_movie = movie_to_idx['Total Recall (1990)']
rec_vector = als_model.item_factors[rec_movie]
np.dot(my_vector, rec_vector)

0.44865954

In [39]:
favorite_movie = 'Toy Story (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 1.0000001),
 (50, 0.81580144),
 (4, 0.57780755),
 (322, 0.5697089),
 (33, 0.5659584),
 (110, 0.5346191),
 (330, 0.44555378),
 (10, 0.41780564),
 (255, 0.4098304),
 (20, 0.40548825),
 (126, 0.36040795),
 (545, 0.3546694),
 (160, 0.342278),
 (32, 0.34150714),
 (34, 0.3369876)]

In [40]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)",
 'Pleasantville (1998)',
 'Shakespeare in Love (1998)',
 'Santa Clause, The (1994)',
 'Forrest Gump (1994)',
 'Hercules (1997)',
 'Mulan (1998)']

In [41]:
movie_recommended = als_model.recommend(6038, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1143, 1.2028604),
 (83, 1.1320685),
 (1176, 1.1305707),
 (1412, 1.0603579),
 (1000, 1.049666),
 (178, 1.0434381),
 (868, 0.98860437),
 (169, 0.9149473),
 (571, 0.9048482),
 (243, 0.9047517),
 (1363, 0.9046188),
 (487, 0.8968121),
 (1512, 0.88817),
 (784, 0.8875634),
 (481, 0.8435272),
 (231, 0.82604754),
 (203, 0.82109874),
 (1156, 0.8109729),
 (1734, 0.7940129),
 (1286, 0.78574276)]

In [42]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Do the Right Thing (1989)',
 'Gandhi (1982)',
 'Sex, Lies, and Videotape (1989)',
 'Right Stuff, The (1983)',
 'Roger & Me (1989)',
 'Unforgiven (1992)',
 'Heavenly Creatures (1994)',
 'Fish Called Wanda, A (1988)',
 'Being There (1979)',
 'Ghostbusters (1984)',
 'Drugstore Cowboy (1989)',
 'Boat, The (Das Boot) (1981)',
 'Buena Vista Social Club (1999)',
 '400 Blows, The (Les Quatre cents coups) (1959)',
 'Kolya (1996)',
 'Insider, The (1999)',
 'Delicatessen (1991)',
 'Big Chill, The (1983)',
 'Three Colors: Blue (1993)',
 'Strangers on a Train (1951)']

In [43]:
lion = movie_to_idx['Lion King, The (1994)']
explain = als_model.explain(6038, csr_data, itemid=lion)

In [44]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.08991986392361244),
 ('Wizard of Oz, The (1939)', 0.0721982993894074),
 ("Singin' in the Rain (1952)", 0.04832004962027704),
 ('Philadelphia (1993)', 0.040526867117467065),
 ('Fugitive, The (1993)', 0.039600090048761194),
 ('West Side Story (1961)', 0.03008083017676018),
 ('Producers, The (1968)', 0.028001385068957524),
 ('Birdy (1984)', 0.026108994543201567),
 ('Three Colors: Red (1994)', 0.025968531781844162),
 ('Apollo 13 (1995)', 0.02401007847835309)]

# 후기
* 추천알고리즘 중 간단 구조의 MF을 사용해봄
* 생각보다 높은 정확도에 놀람
* 평점에 따른 가중치를 어떻게 둬야할지 모르겠다